In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
from src.paychex_ml.utils import load_credentials
from src.paychex_ml.utils import get_blob_list
from src.paychex_ml.utils import upload_df_parquet
from src.paychex_ml.upload_data import upload_data
from src.paychex_ml.clean_data import get_df
from src.paychex_ml.clean_data import join_all

In [4]:
credentials = load_credentials("blob_storage",
                               file="C:/Users/bruno.gonzalez/DataspellProjects/Paychex_revenue_forecast/credentials.yml")

In [5]:
# Start client
container_name = "raw-data"
blob_service_client = BlobServiceClient.from_connection_string(credentials['conn_string'])

In [8]:
blob_list = get_blob_list(blob_service_client)

In [9]:
blob_list

['401kRevenue.txt',
 '401kRevenueDetail.txt',
 'ASORevenue.txt',
 'ASORevenueDrivers.txt',
 'BlendedProductRevenue.txt',
 'BlendedProductRevenueDrivers.txt',
 'IFHC.txt',
 'InsuranceRevenue.txt',
 'InsuranceRevenueDetail.txt',
 'InternationalRevenue.txt',
 'InternationalRevenueStats.txt',
 'OasisASORevenueDetail.txt',
 'OnlineRevenue.txt',
 'OnlineRevenueDetail.txt',
 'OtherMgmtRevenue.txt',
 'OtherMgmtRevenueDetail.txt',
 'PEORevenue.txt',
 'PEORevenueDetail.txt',
 'PEORevenueDetailDrivers.txt',
 'PayrollSurePayrollASOInternationalHighLevelRevenue.txt',
 'SurePayollRevenue.txt',
 'SurePayollRevenueDrivers.txt',
 'W2DelOtherRevenue.txt',
 'W2DelOtherRevenueDrivers.txt']

## 11 Payroll blended products

In [70]:
blob_list[4]

'BlendedProductRevenue.txt'

In [77]:
df = get_df(blob_service_client, blob_list[4], container= container_name)

In [78]:
df

Total Activity  \
                                      Total Flex Blended Products Revenue   
                                                               Auburn RSC   
                                                    FEE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                         122970372.0   
                            \nJun                               9624260.0   
                            \nJul                              11784050.0   
                            \nAug                               9592805.0   
                            \nSep                               9327331.0   
...                                                                   ...   
Forecast 8+4           FY22 \nJan                                     0.0   
                            \nFeb                                     0.0   
                            \nMar                                     0.0   
                            \nApr                                     0.0   
                            \nMay                                     0.0   

                                                                        \
                                                                         
                                                                         
                                      FEE REVENUE - CONTRACT PLAN - RW   
Actual   Final Revised FY15 YearTotal                              0.0   
                            \nJun                                  0.0   
                            \nJul                                  0.0   
                            \nAug                                  0.0   
                            \nSep                                  0.0   
...                                                                ...   
Forecast 8+4           FY22 \nJan                             259200.0   
                            \nFeb                             135806.0   
                            \nMar                             182737.0   
                            \nApr                             144785.0   
                            \nMay                             138549.0   

                                                               \
                                                                
                                                                
                                      OTHER REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                     0.0   
                            \nJun                         0.0   
                            \nJul                         0.0   
                            \nAug                         0.0   
                            \nSep                         0.0   
...                                                       ...   
Forecast 8+4           FY22 \nJan                     10350.0   
                            \nFeb                         0.0   
                            \nMar                         0.0   
                            \nApr                         0.0   
                            \nMay                         0.0   

                                                                     \
                                                                      
                                                                      
                                      MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                           0.0   
                            \nJun                               0.0   
                            \nJul                               0.0   
                            \nAug                               0.0   
                            \nSep                               0.0   
...                                                             ...   
Forecast 8+4           FY22 \nJan                         8642359.0   
                            \nFeb  

In [79]:
df.loc[("Forecast","6+6","FY22","YearTotal"),:].groupby(level=2).sum()/1000000

5060 STATE UNEMPLOYMENT INSURANCE      5.730839
5062 SUI ADMIN                        16.549797
ADVANTAGE/PACE                        27.913207
Albuquerque RSC                       45.141791
Auburn RSC                            79.630992
Baton Rouge RSC                       32.903493
CONS ASSOCIATES                        0.000000
CONS HIGH LEVEL and ACCRUALS           7.580981
CONS Multi Product Services          227.874975
DSC/Online                           377.425076
Denver RSC                            39.073053
Greensboro RSC                        39.353555
Indianapolis RSC                      53.955133
Lehigh Valley RSC                    118.754871
Middletown RSC                        26.472856
Nashville RSC                         25.335217
Phoenix RSC                           81.154682
Prior Year Adjustment                  0.000000
Rochester RSC                         79.902564
TOTAL LOGISTICS                        0.000000
TOTAL OPERATIONS CC                    0

In [80]:
(df.loc[("Forecast","6+6","FY22","YearTotal"),:].groupby(level=2).sum()/1000000).sum()

1327.136744

In [85]:
df_11 = df.groupby(level=2, axis=1).sum().stack()
df_11

Actual    Final Revised  FY15  YearTotal  5060 STATE UNEMPLOYMENT INSURANCE     36844499.0
                                          5062 SUI ADMIN                               0.0
                                          ADVANTAGE/PACE                        36018424.0
                                          Albuquerque RSC                       71031512.0
                                          Auburn RSC                           102981785.0
                                                                                  ...     
Forecast  8+4            FY22  \nMay      Prior Year Adjustment                        0.0
                                          Rochester RSC                          6630433.0
                                          TOTAL LOGISTICS                              0.0
                                          TOTAL OPERATIONS CC                          0.0
                                          Tampa RSC                              3078974.0

In [100]:
df_11.loc["Forecast"]["6+6"]["FY22"]["YearTotal"].sum()/1000000

1327.136744

## 12, 13, 14, 15, 31

In [107]:
blob_list[19]

'PayrollSurePayrollASOInternationalHighLevelRevenue.txt'

In [108]:
df = get_df(blob_service_client, blob_list[19], container= container_name)

In [109]:
df = df.loc[:,
     (
         ["period", "Total Activity"],
         ['Total Delivery Revenue','Total Other Processing Revenue','Total W-2 Revenue',
          'HR Solutions (excl PEO)','Total HR Solutions/ASO (Payroll side)',''],
         ['TOTAL OPERATIONS','CONS HRS', 'Prior Year Adjustment', 'Total Paychex','']
     )
     ]

In [110]:
df

Total Activity  \
                                          Total Delivery Revenue   
                                                TOTAL OPERATIONS   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal               1.186875e+08   
                            \nJun                   7.393352e+06   
                            \nJul                   1.351374e+07   
                            \nAug                   7.286967e+06   
                            \nSep                   7.081862e+06   
...                                                          ...   
Forecast 8+4           FY22 \nJan                   2.243552e+07   
                            \nFeb                   6.683674e+06   
                            \nMar                   7.556914e+06   
                            \nApr                   1.094369e+07   
                            \nMay                   7.961770e+06   

                                                                  \
                                                                   
                                                        CONS HRS   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                        0.0   
                            \nJun                            0.0   
                            \nJul                            0.0   
                            \nAug                            0.0   
                            \nSep                            0.0   
...                                                          ...   
Forecast 8+4           FY22 \nJan                            0.0   
                            \nFeb                            0.0   
                            \nMar                            0.0   
                            \nApr                            0.0   
                            \nMay                            0.0   

                                                                      \
                                      Total Other Processing Revenue   
                                                    TOTAL OPERATIONS   
                                          Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                    63488855.55   
                            \nJun                         4765775.19   
                            \nJul                         6502729.54   
                            \nAug                         4215268.17   
                            \nSep                         4726806.24   
...                                                              ...   
Forecast 8+4           FY22 \nJan                        20866313.40   
                            \nFeb                        14110893.97   
                            \nMar                        12680918.38   
                            \nApr                        10225582.63   
                            \nMay                         6506572.85   

                                                                  \
                                                                   
                                           Prior Year Adjustment   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                        0.0   
                            \nJun                            0.0   
                            \nJul                            0.0   
                            \nAug                            0.0   
                            \nSep                            0.0   
...                                                          ...   
Forecast 8+4           FY22 \nJan                            0.0   
                            \nFeb                            0.0   
                            \nMar                            0.0   
                            \nApr          

In [113]:
df.loc[("Forecast","6+6","FY22","YearTotal"),:].groupby(level=1).sum()/1000000

HR Solutions (excl PEO)                  598.036988
Total Delivery Revenue                   117.029362
Total HR Solutions/ASO (Payroll side)    210.493782
Total Other Processing Revenue           102.279869
Total W-2 Revenue                         81.781030
Name: (Forecast, 6+6, FY22, YearTotal), dtype: float64

In [116]:
df.loc[("Forecast","6+6","FY22","YearTotal"),:].groupby(level=[1,2]).sum()/1000000

HR Solutions (excl PEO)                Total Paychex            598.036988
Total Delivery Revenue                 CONS HRS                   0.000000
                                       TOTAL OPERATIONS         117.029362
Total HR Solutions/ASO (Payroll side)  CONS HRS                   0.000000
                                       TOTAL OPERATIONS         210.493782
Total Other Processing Revenue         Prior Year Adjustment      0.000000
                                       TOTAL OPERATIONS         102.279869
Total W-2 Revenue                      Prior Year Adjustment      0.000000
                                       TOTAL OPERATIONS          81.781030
Name: (Forecast, 6+6, FY22, YearTotal), dtype: float64

## 16 Sure payrol

In [117]:
blob_list[20]

'SurePayollRevenue.txt'

In [118]:
df = get_df(blob_service_client, blob_list[20], container= container_name)

In [119]:
df

Total Activity  \
                                                    PR PROC   
                                          1501 SP Direct GB   
                                      FEE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal           21219332.78   
                            \nJun                1716994.82   
                            \nJul                2088717.79   
                            \nAug                1411140.00   
                            \nSep                1729748.29   
...                                                     ...   
Forecast 8+4           FY22 \nJan                -154767.08   
                            \nFeb                      0.00   
                            \nMar                      0.00   
                            \nApr                      0.00   
                            \nMay                      0.00   

                                                               \
                                                                
                                                                
                                      OTHER REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                    0.00   
                            \nJun                        0.00   
                            \nJul                        0.00   
                            \nAug                        0.00   
                            \nSep                        0.00   
...                                                       ...   
Forecast 8+4           FY22 \nJan                    -2397.90   
                            \nFeb                  -159107.24   
                            \nMar                  -160318.79   
                            \nApr                  -162333.29   
                            \nMay                  -164148.24   

                                                                     \
                                                                      
                                                                      
                                      MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                          0.00   
                            \nJun                              0.00   
                            \nJul                              0.00   
                            \nAug                              0.00   
                            \nSep                              0.00   
...                                                             ...   
Forecast 8+4           FY22 \nJan                         -14888.45   
                            \nFeb                              0.00   
                            \nMar                              0.00   
                            \nApr                              0.00   
                            \nMay                              0.00   

                                                                 \
                                                                  
                                                                  
                                      SALES DISCOUNTS - NA - RW   
Actual   Final Revised FY15 YearTotal                       0.0   
                            \nJun                           0.0   
                            \nJul                           0.0   
                            \nAug                           0.0   
                            \nSep                           0.0   
...                                                         ...   
Forecast 8+4           FY22 \nJan                     -116200.4   
                            \nFeb                           0.0   
                            \nMar                           0.0   
                            \nApr                           0.0   
                            \nMay                           0.0   

                                                   

In [122]:
(df.loc[("Forecast","6+6","FY22", "YearTotal"),
       ("Total Activity")]).sum()/1000000

97.27571197

In [125]:
df.loc[("Forecast","6+6","FY22", "YearTotal"),
       ("Total Activity")].groupby(level=1).sum()

1500 SP DIRECT                     0.00
1501 SP Direct GB           51105717.36
1502 SP ETAP                18961029.34
1504 SP CITIBANK                   0.00
1506 SP BOP PARTNER          3455330.61
1507 SP NORTHWEST MUTUAL           0.00
1508 SP WELLS FARGO          2205497.50
1509 SP SUN TRUST            6451943.12
1510 SP HUNTINGTON            693804.08
1511 SP UBOC                  695762.30
1513 SP SITTER BANK                0.00
1514 SP CITIBANK             2624912.21
1517 SP MT BANK               369394.92
1518 SP Direct HH           10712320.53
Name: (Forecast, 6+6, FY22, YearTotal), dtype: float64

In [128]:
df_16 = df.groupby(level=2, axis=1).sum().stack()
df_16

Actual    Final Revised  FY15  YearTotal  1500 SP DIRECT                0.00
                                          1501 SP Direct GB      25466909.19
                                          1502 SP ETAP            2928476.25
                                          1504 SP CITIBANK              0.00
                                          1506 SP BOP PARTNER     1437100.99
                                                                    ...     
Forecast  8+4            FY22  \nMay      1511 SP UBOC              55971.54
                                          1513 SP SITTER BANK           0.00
                                          1514 SP CITIBANK         211765.95
                                          1517 SP MT BANK           30950.00
                                          1518 SP Direct HH        812514.47
Length: 2548, dtype: float64

In [129]:
df_16.loc["Forecast"]["6+6"]["FY22"]["YearTotal"].sum()/1000000

97.27571197

## 17 International

In [130]:
blob_list[9]

'InternationalRevenue.txt'

In [131]:
df = get_df(blob_service_client, blob_list[9], container= container_name)

In [132]:
df

Total Activity  \
                                                           PR PROC   
                                      3000 GERMANY GMBH OPERATIONS   
                                             FEE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                   4309124.40   
                            \nJun                        369471.91   
                            \nJul                        401379.94   
                            \nAug                        365608.89   
                            \nSep                        314304.69   
...                                                            ...   
Forecast 8+4           FY22 \nJan                         24115.76   
                            \nFeb                          9530.09   
                            \nMar                          8902.97   
                            \nApr                          7912.84   
                            \nMay                          7506.39   

                                                               \
                                                                
                                                                
                                      OTHER REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                    0.00   
                            \nJun                        0.00   
                            \nJul                        0.00   
                            \nAug                        0.00   
                            \nSep                        0.00   
...                                                       ...   
Forecast 8+4           FY22 \nJan                   107274.28   
                            \nFeb                    20647.05   
                            \nMar                    91301.90   
                            \nApr                        0.00   
                            \nMay                        0.00   

                                                                     \
                                                                      
                                                                      
                                      MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                          0.00   
                            \nJun                              0.00   
                            \nJul                              0.00   
                            \nAug                              0.00   
                            \nSep                              0.00   
...                                                             ...   
Forecast 8+4           FY22 \nJan                         547429.63   
                            \nFeb                         496008.45   
                            \nMar                         531171.41   
                            \nApr                         522273.28   
                            \nMay                         524802.35   

                                                                 \
                                                                  
                                                                  
                                      SALES DISCOUNTS - NA - RW   
Actual   Final Revised FY15 YearTotal                  -2771.56   
                            \nJun                       -147.56   
                            \nJul                       -278.50   
                            \nAug                       -188.61   
                            \nSep                       -301.66   
...                                                         ...   
Forecast 8+4           FY22 \nJan                          0.00   
                            \nFeb                          0.00   
                            \nMar                          0.00   
                            \nApr                          0.00   
                     

In [133]:
(df.loc[("Forecast","6+6","FY22", "YearTotal"),
        ("Total Activity")]).sum()/1000000

54.453345230000004

In [134]:
df.loc[("Forecast","6+6","FY22", "YearTotal"),
       ("Total Activity")].groupby(level=1).sum()

3000 GERMANY GMBH OPERATIONS                  6831282.48
3010 GERMANY LOHNDATA OPERATIONS              6436879.65
3011 GERMANY LOHNDATA SOFTWARE OPERATIONS      353663.30
3300 Lessor Group Aps                               0.00
3320 Lessor Suite                            17959905.73
3321 Danske Loensystemer A/S                 18095983.52
3322 iLohngehalt Internet Serivces GmbH         51149.55
3323 Lorloenn A/S                                   0.00
3324 Swelon A/S                                  2024.36
3325 Dynamics Suite                                 0.00
3360 Emply Operations                               0.00
3370 Emply ApS International Ops              4722456.64
Name: (Forecast, 6+6, FY22, YearTotal), dtype: float64

In [135]:
df_17 = df.groupby(level=2, axis=1).sum().stack()
df_17

Actual    Final Revised  FY15  YearTotal  3000 GERMANY GMBH OPERATIONS                 4306352.84
                                          3010 GERMANY LOHNDATA OPERATIONS             4638931.53
                                          3011 GERMANY LOHNDATA SOFTWARE OPERATIONS     553231.73
                                          3300 Lessor Group Aps                              0.00
                                          3320 Lessor Suite                                  0.00
                                                                                          ...    
Forecast  8+4            FY22  \nMay      3323 Lorloenn A/S                                  0.00
                                          3324 Swelon A/S                                    0.00
                                          3325 Dynamics Suite                                0.00
                                          3360 Emply Operations                              0.00
                    

In [136]:
df_17.loc["Forecast"]["6+6"]["FY22"]["YearTotal"].sum()/1000000

54.45334523

## 401 Revenue

In [146]:
blob_list[0]

'401kRevenue.txt'

In [147]:
df = get_df(blob_service_client, blob_list[0], container= container_name)

In [148]:
df

Total Activity  \
                                              Total 401k Revenue   
                                                   Total Paychex   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                200963139.0   
                            \nJun                     14883414.0   
                            \nJul                     15071771.0   
                            \nAug                     14783933.0   
                            \nSep                     17075301.0   
...                                                          ...   
Forecast 8+4           FY22 \nJan                     25826261.0   
                            \nFeb                     24740442.0   
                            \nMar                     25937318.0   
                            \nApr                     25600810.0   
                            \nMay                     26080025.0   

                                                                           \
                                                                            
                                                                            
                                      DIRECT BILL/MONEY MGMT REVENUE - RW   
Actual   Final Revised FY15 YearTotal                          53868722.0   
                            \nJun                               4261452.0   
                            \nJul                               4513388.0   
                            \nAug                               4486437.0   
                            \nSep                               4355892.0   
...                                                                   ...   
Forecast 8+4           FY22 \nJan                               9956103.0   
                            \nFeb                               8475391.0   
                            \nMar                               9403945.0   
                            \nApr                               9162176.0   
                            \nMay                               9516061.0   

                                               NaN         Total Activity  \
                                               NaN     Total 401k Revenue   
                                               NaN CONSOLIDATED 401K-S125   
                                      *fee revenue             Sales - RW   
Actual   Final Revised FY15 YearTotal  147094416.0                14400.0   
                            \nJun       10621962.0                 1229.0   
                            \nJul       10558383.0                  832.0   
                            \nAug       10297496.0                  986.0   
                            \nSep       12719410.0                 1326.0   
...                                            ...                    ...   
Forecast 8+4           FY22 \nJan       15870158.0                 3311.0   
                            \nFeb       16265051.0                 1852.0   
                            \nMar       16533373.0                 2097.0   
                            \nApr       16438635.0                 1811.0   
                            \nMay       16563964.0                 2001.0   

                                                                             \
                                                                              
                                      Total Paychex                           
                                        Losses - RW Ending Client Base - RW   
Actual   Final Revised FY15 YearTotal      -10554.0                 69512.0   
                            \nJun            -575.0                 66475.0   
                            \nJul            -956.0                 66343.0   
                            \nAug            -550.0                 66768.0   
                            \nSep            -617.0                 67466.0   
...   

In [149]:
df.loc[("Forecast","6+6","FY22","YearTotal"),
         ("Total Activity","Total 401k Revenue", "Total Paychex", "Total Service Revenue - RW")]/1000000

306.22887

In [153]:
df.loc[("Forecast","6+6","FY22", "YearTotal"),
       ("Total Activity")].groupby(level=[1,2]).sum()

CONSOLIDATED 401K-S125  Sales - RW                                  2.253900e+04
Total Paychex           Average Asset Values - Calculated - RW      4.817152e+10
                        Average Assets/Clients - Calculated - RW    4.817600e+05
                        Basis Points - RW                           7.000000e+00
                        Basis Points - Solicitor - RW              -1.000000e+00
                        Basis Pts - Calc - RW                       0.000000e+00
                        DIRECT BILL/MONEY MGMT REVENUE - RW         1.165454e+08
                        Ending Asset Values - RW                    4.985099e+10
                        Ending Client Base - RW                     1.045070e+05
                        Losses - RW                                -1.354400e+04
                        Maintenance Rate - Calculated - RW          1.200000e+02
                        Set Up Rate - Calculated - RW               5.380000e+02
                        Tota

# Join Tables

In [145]:
pd.concat([df_11, df_16, df_17], axis=1)\
    .rename(columns={'1': 'Payroll Blende products', '2': 'SurePayrol', '3': 'Total International'})

0  \
Actual Final Revised FY15 \nApr     1500 SP DIRECT                NaN   
                                    1501 SP Direct GB             NaN   
                                    1502 SP ETAP                  NaN   
                                    1504 SP CITIBANK              NaN   
                                    1506 SP BOP PARTNER           NaN   
...                                                               ...   
Plan   Final Revised FY22 YearTotal Prior Year Adjustment         0.0   
                                    Rochester RSC          74609684.0   
                                    TOTAL LOGISTICS               0.0   
                                    TOTAL OPERATIONS CC           0.0   
                                    Tampa RSC              44055204.0   

                                                                    1   2  
Actual Final Revised FY15 \nApr     1500 SP DIRECT               0.00 NaN  
                                    1501 SP Direct GB      2154679.01 NaN  
                                    1502 SP ETAP            289900.11 NaN  
                                    1504 SP CITIBANK             0.00 NaN  
                                    1506 SP BOP PARTNER     140454.76 NaN  
...                                                               ...  ..  
Plan   Final Revised FY22 YearTotal Prior Year Adjustment         NaN NaN  
                                    Rochester RSC                 NaN NaN  
                                    TOTAL LOGISTICS               NaN NaN  
                                    TOTAL OPERATIONS CC           NaN NaN  
                                    Tampa RSC                     NaN NaN  

[8736 rows x 3 columns]